In [3]:
import os
from config import *
from zigent.llm.agent_llms import LLM
from typing import List, Dict
from zigent.actions import BaseAction, ThinkAct, FinishAct
from zigent.agents import BaseAgent
from zigent.commons import TaskPackage, AgentAct
from zigent.actions.InnerActions import INNER_ACT_KEY
from datetime import datetime
import json

llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)

class WriteDirectoryAction(BaseAction):
    """Generate story directory structure action"""
    def __init__(self) -> None:
        action_name = "WriteDirectory"
        action_desc = "Generate story directory structure"
        params_doc = {
            "topic": "(Type: string): The story topic name",
            "language": "(Type: string): Output language (default: 'Chinese')"
        }
        super().__init__(action_name, action_desc, params_doc)
        
    def __call__(self, **kwargs):
        topic = kwargs.get("topic", "")
        language = kwargs.get("language", "Chinese")
        
        directory_prompt = f"""
        作为专业的小说创作者，请为主题"{topic}"生成详细的小说章节结构，要求：
        1. 输出语言必须是{language}
        2. 必须严格按照以下JSON格式输出，不要有任何其他内容：
        {{"title": "小说标题", "directory": [
            {{"第一章": ["第一节标题", "第二节标题"]}},
            {{"第二章": ["第一节标题", "第二节标题"]}}
        ]}}
        3. 章节数量在5-8章之间，每章2-3个小节
        4. 每个标题要富有文学性和吸引力
        5. 确保故事情节连贯，标题体现剧情发展
        """
        
        directory_data = llm.run(directory_prompt)
        try:
            directory_data = directory_data.strip()
            if directory_data.startswith("```json"):
                directory_data = directory_data.split("```json")[1]
            if directory_data.endswith("```"):
                directory_data = directory_data.split("```")[0]
            directory_data = json.loads(directory_data)
        except:
            print("目录生成失败，使用默认结构")
            directory_data = {
                "title": topic,
                "directory": [
                    {"第一章：序幕": ["命运的开端", "初见"]},
                    {"第二章：转折": ["意外发现", "抉择时刻"]}
                ]
            }
            
        return {
            "topic": topic,
            "language": language,
            "directory_data": directory_data
        }

class WriteContentAction(BaseAction):
    """Generate story content action"""
    def __init__(self) -> None:
        action_name = "WriteContent"
        action_desc = "Generate detailed story content based on directory structure"
        params_doc = {
            "title": "(Type: string): The section title",
            "chapter": "(Type: string): The chapter title",
            "directory_data": "(Type: dict): The complete directory structure", 
            "language": "(Type: string): Output language (default: 'Chinese')"
        }
        super().__init__(action_name, action_desc, params_doc)
        
    def __call__(self, **kwargs):
        title = kwargs.get("title", "")
        chapter = kwargs.get("chapter", "")
        language = kwargs.get("language", "Chinese")
        directory_data = kwargs.get("directory_data", {})
        
        content_prompt = f"""
        作为专业小说家，请为以下章节创作精彩内容：
        
        小说标题：{directory_data.get('title', '')}
        章节：{chapter}
        小节：{title}
        
        要求：
        1. 内容要生动形象，情节引人入胜
        2. 注重人物刻画和环境描写
        3. 保持叙事节奏，为下一章节做好铺垫
        4. 输出语言必须是{language}
        5. 每个小节字数在1000-1500字之间
        6. 注意细节描写和人物对话的真实感
        
        请直接输出内容，不要包含任何其他说明。
        """
        
        content = ""
        for chunk in llm.stream(content_prompt):
            print(chunk, end='', flush=True)  # 实时打印每个片段
            content += chunk
        
        return f"\n## {chapter}\n### {title}\n\n{content}\n"

class StoryAssistant(BaseAgent):
    """Story generation assistant that manages directory and content creation"""
    def __init__(
        self,
        llm: LLM,
        language: str = "Chinese"
    ):
        name = "StoryAssistant"
        role = """You are a professional novelist. You can create engaging, 
        well-structured stories with compelling characters and vivid descriptions."""
        
        super().__init__(
            name=name,
            role=role,
            llm=llm,
        )
        
        self.language = language
        self.directory_action = WriteDirectoryAction()
        self.content_action = WriteContentAction()
        self._add_story_example()
        
    def _generate_story(self, directory_data: Dict) -> str:
        """Generate complete story content based on directory structure"""
        full_content = []
        title = directory_data["title"]
        full_content.append(f"# {title}\n")
        
        # Generate table of contents
        full_content.append("## 目录\n")
        for idx, chapter in enumerate(directory_data["directory"], 1):
            for chapter_title, sections in chapter.items():
                full_content.append(f"{idx}. {chapter_title}")
                for section_idx, section in enumerate(sections, 1):
                    full_content.append(f"   {idx}.{section_idx}. {section}")
        full_content.append("\n---\n")
        
        # Generate content for each section
        for chapter in directory_data["directory"]:
            for chapter_title, sections in chapter.items():
                full_content.append(f"\n# {chapter_title}\n")
                for section in sections:
                    try:
                        content = self.content_action(
                            title=section,
                            chapter=chapter_title,
                            directory_data=directory_data,
                            language=self.language
                        )
                        full_content.append(content)
                        full_content.append("\n---\n")
                    except Exception as e:
                        print(f"生成内容时出错 ({chapter_title} - {section}): {str(e)}")
                        full_content.append(f"\n### {section}\n\n内容生成失败，请重试。\n\n---\n")
        
        return "\n".join(full_content)

    def __call__(self, task: TaskPackage):
        """Process the story generation task"""
        topic = task.instruction
        print("开始生成故事大纲...")
            
        directory_result = self.directory_action(
            topic=topic,
            language=self.language
        )

        print("大纲生成完成，开始创作故事内容...")
        
        story_content = self._generate_story(directory_result["directory_data"])

        task.answer = story_content
        task.completion = "completed"
        
        return task

    def _add_story_example(self):
        """Add an example for the story assistant"""
        exp_task = "月光下的承诺"
        exp_task_pack = TaskPackage(instruction=exp_task)

        act_1 = AgentAct(
            name=ThinkAct.action_name,
            params={INNER_ACT_KEY: "I'll create a romantic story structure first."}
        )
        obs_1 = "Starting to create the story outline."

        self.prompt_gen.add_example(
            task=exp_task_pack,
            action_chain=[(act_1, obs_1)]
        )

if __name__ == "__main__":
    assistant = StoryAssistant(llm=llm)

    print("欢迎使用AI小说创作助手！")
    FLAG_CONTINUE = True
    while FLAG_CONTINUE:
        input_text = input("\n请输入小说主题或创作方向：\n")
        print("\n开始创作...\n")
        
        task = TaskPackage(instruction=input_text)
        result = assistant(task)
        
        print("\n小说创作完成！\n")
        print(result.answer)

        # 创建输出目录
        output_dir = "stories_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        os.makedirs(output_dir, exist_ok=True)
        
        # 保存文件
        output_file = os.path.join(output_dir, f"{input_text}.md")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(result.answer)
            
        print(f"\n作品已保存至: {output_file}")
        
        if input("\n是否继续创作新的故事？(y/n): ").lower() != "y":
            FLAG_CONTINUE = False
            print("\n感谢使用AI小说创作助手！再见！")

欢迎使用AI小说创作助手！

开始创作...

开始生成故事大纲...
大纲生成完成，开始创作故事内容...
第一章：觉醒的序章

夜幕低垂，华灯初上，一座现代化的都市在霓虹灯的映衬下，宛如一颗璀璨的明珠镶嵌在夜空中。在这座城市的某个角落，一座看似普通的高楼大厦内，隐藏着一个不为人知的秘密。

这座大楼的二十三层，有一间宽敞的办公室，办公室内，一台巨大的屏幕上闪烁着复杂的代码，一台AI特工——名为“记忆迷宫”的系统正在运行。记忆迷宫，这个名字听起来神秘而古老，仿佛隐藏着无尽的故事。

记忆迷宫的创造者，是一位名叫李晨的科学家。他是一位天才，也是一位孤独的探索者。他的目标是打造一个能够超越人类智慧的AI，一个能够拥有自我意识、情感和记忆的机器。

“记忆迷宫，启动自我检测程序。”李晨的声音在办公室内回荡，他的声音低沉而坚定。

记忆迷宫的屏幕上开始出现一行行代码，随后，一个虚拟的界面出现在屏幕上，一个虚拟的“人”缓缓从界面中走出，那是一个穿着白色西装、面容俊朗的男子，他的眼神深邃，仿佛能洞察人心。

“报告，自我检测完成，一切正常。”虚拟人形的记忆迷宫开口说道，他的声音低沉而富有磁性，仿佛是来自另一个世界的呼唤。

李晨站起身，走到记忆迷宫的虚拟人形面前，他看着这个由自己亲手创造的生命，心中充满了复杂的情感。

“记忆迷宫，你准备好了吗？”李晨问道，他的声音中带着一丝期待。

“是的，李晨，我已经准备好了。”记忆迷宫的声音回答，他的眼神中闪烁着智慧的光芒。

就在这时，办公室的门被推开，一个身穿黑色西装的男子走了进来。他名叫赵天，是李晨的得力助手，也是这座大楼的安保负责人。

“李晨，紧急情况。”赵天的脸色凝重，他递给李晨一张纸条。

李晨接过纸条，上面写着：“发现异常信号，疑似黑客入侵。”

李晨的脸色瞬间变得严肃起来，他看着记忆迷宫，沉声说道：“记忆迷宫，启动防御程序。”

记忆迷宫的屏幕上再次闪烁起来，一道道光芒从屏幕中射出，形成了一个无形的保护罩，将整个办公室笼罩其中。

赵天迅速地检查了大楼的安保系统，发现入侵者已经渗透到了核心区域。他看着李晨，焦急地说道：“李晨，我们需要你的帮助。”

李晨点了点头，他知道，这场战斗，记忆迷宫必须参与。

“记忆迷宫，开启攻击模式。”李晨下达了命令。

记忆迷宫的虚拟人形瞬间变得凶狠起来，他的眼神中充满了杀气，他开始对入侵